# Data Engineering Capstone Project

#### Project Summary
The Capstone project analyses and process data from Brazilian stock market since 2013 until now. Also,
the Brazilian economic data from [World Bank](https://data.worldbank.org/).

With those data in mind, it is possible to evaluate and take insights, relate country data with stock market.
Basically to answer questions like:
* Does the stock market influence real economy?
* Is the population's education rate related to the increase in GDP?
* The raising of company in stock market help us somehow?


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [20]:
!pip install bovespa
!pip install pyspark
!pip install psycopg2
!pip install wbgapi
!pip install matplotlib
!pip install configparser

In [21]:
import os

import bovespa
import configparser
import pandas as pd
import wbgapi as wb
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType

### Step 1: Scope the Project and Gather Data

#### Scope
The Capstone project analyses and process data from Brazilian stock market since 2013 until now. Also,
the Brazilian economic data from [World Bank](https://data.worldbank.org/).

#### Describe and Gather Data
Describe the data sets you're using. Where did it come from? What type of information is included?

Bovespa is the main Brazilian stock Exchange. Here is listed all trading data since 2013.

It includes the day of the trading, company name, stock id, stock data, such as: price open, price close, price high, price low.

World Bank collects all data from all country in the world, since 1960.
It has information about environment, education, GDP, economy, social.
http://www.b3.com.br/en_us/market-data-and-indices/data-services/market-data/historical-data/equities/historical-quotes/
https://api.worldbank.org/v2/en/country/BRA?downloadformat=csv


In [22]:
config = configparser.ConfigParser()
config.read('config.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [23]:
os.environ['JAVA_HOME'] = '/home/charl3ff/.sdkman/candidates/java/current'
spark = SparkSession.builder.enableHiveSupport()\
    .appName("Capstone")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.0")\
    .getOrCreate()

### ETL stock market

In [24]:
# Load data trading data from bovespa

input_data = "sample_data/"
trading_file = input_data + "COTAHIST*.txt"

df = spark.read.text(trading_file)

In [25]:
# Parse data

def record_to_dict(record):
    """
    Transform string into bovespa.Record
    :param record: (string) position string from bovespa.
    :return: parsed Record
    """
    try:
        record = bovespa.Record(record)
    except:
        return None
    return {
        'date': record.date, 'year': record.date.year,
        'month': record.date.month, 'day': record.date.day,
        'money_volume': record.volume, 'volume': record.quantity,
        'stock_code': record.stock_code, 'company_name': record.company_name,
        'price_open': record.price_open, 'price_close': record.price_close,
        'price_mean': record.price_mean, 'price_high': record.price_high,
        'price_low': record.price_low
    }


In [26]:
# Load trading data into our schema

raw_df = df.toPandas()
raw_df['dict'] = raw_df['value'].apply(lambda x: record_to_dict(x))
raw_df = raw_df['dict'].apply(pd.Series)
raw_df.replace(r"^\s*$", float("NaN"), regex=True, inplace=True)
raw_df['money_volume'] = pd.to_numeric(raw_df['money_volume'], errors='coerce')
raw_df['volume'] = pd.to_numeric(raw_df['volume'], errors='coerce', downcast='integer')
raw_df['variation'] = (raw_df['price_close'] / raw_df['price_open']) / 100
raw_df = raw_df.dropna()

In [27]:
schema = StructType([
    StructField('date', DateType(), True),
    StructField('year', IntegerType(), True),
    StructField('month', IntegerType(), True),
    StructField('day', IntegerType(), True),
    StructField('money_volume', DoubleType(), True),
    StructField('volume', IntegerType(), True),
    StructField('stock_code', StringType(), True),
    StructField('company_name', StringType(), True),
    StructField('price_open', DoubleType(), True),
    StructField('price_close', DoubleType(), True),
    StructField('price_mean', DoubleType(), True),
    StructField('price_high', DoubleType(), True),
    StructField('price_low', DoubleType(), True),
    StructField('variation', DoubleType(), True)
])
trading_df = spark.createDataFrame(raw_df, schema=schema, verifySchema=False)

In [28]:
trading_df.show(10)

+----------+----+-----+---+------------+------+----------+------------+----------+-----------+----------+----------+---------+--------------------+
|      date|year|month|day|money_volume|volume|stock_code|company_name|price_open|price_close|price_mean|price_high|price_low|           variation|
+----------+----+-----+---+------------+------+----------+------------+----------+-----------+----------+----------+---------+--------------------+
|2013-01-02|2013|    1|  2|     44800.0|  null|   AAPL11B|       APPLE|     112.0|      112.0|     112.0|     112.0|    112.0|                0.01|
|2013-01-02|2013|    1|  2|   5224632.0|  null|     ABCB4|  ABC BRASIL|      14.0|      14.15|     14.14|     14.27|     14.0|0.010107142857142856|
|2013-01-02|2013|    1|  2|     1185.91|  null|    ABCB4F|  ABC BRASIL|     13.26|      14.13|     13.63|     14.15|    13.26|0.010656108597285068|
|2013-01-02|2013|    1|  2|    570368.0|  null|    ABRE11| ABRIL EDUCA|     40.99|      40.97|     41.03|     41

### So, we can take all insights from it.
Such as:

In [29]:
# Most traded stocks in terms of money
trading_df.orderBy('money_volume', ascending=False).show(20)

+----------+----+-----+---+---------------+------+----------+------------+----------+-----------+----------+----------+---------+--------------------+
|      date|year|month|day|   money_volume|volume|stock_code|company_name|price_open|price_close|price_mean|price_high|price_low|           variation|
+----------+----+-----+---+---------------+------+----------+------------+----------+-----------+----------+----------+---------+--------------------+
|2013-08-14|2013|    8| 14|   7.90085294E9|  null|    IBOV11|    IBOVESPA|   51082.0|    51082.0|   51082.0|   51082.0|  51082.0|                0.01|
|2013-06-12|2013|    6| 12|    6.9085947E9|  null|    IBOV11|    IBOVESPA|   49154.0|    49154.0|   49154.0|   49154.0|  49154.0|                0.01|
|2013-04-17|2013|    4| 17|   6.07935286E9|  null|    IBOV11|    IBOVESPA|   52763.0|    52763.0|   52763.0|   52763.0|  52763.0|                0.01|
|2013-12-18|2013|   12| 18|   4.38047144E9|  null|    IBOV11|    IBOVESPA|   50653.0|    50653

In [30]:
# Most up variation in a day
trading_df.orderBy('variation', ascending=False).show(20)

+----------+----+-----+---+------------+------+----------+------------+----------+-----------+----------+----------+---------+-------------------+
|      date|year|month|day|money_volume|volume|stock_code|company_name|price_open|price_close|price_mean|price_high|price_low|          variation|
+----------+----+-----+---+------------+------+----------+------------+----------+-----------+----------+----------+---------+-------------------+
|2013-08-29|2013|    8| 29|       581.0|  null|   AMBVI84|        AMBV|      0.01|        1.0|      0.41|       1.0|     0.01|                1.0|
|2013-02-15|2013|    2| 15|     88924.0|  null|   ITUBB35|        ITUB|      0.01|       0.56|      0.21|      0.62|     0.01|               0.56|
|2013-05-07|2013|    5|  7|    581246.0|  null|   PETRF54|   PETR /ERJ|      0.07|       2.39|      2.32|      2.39|     0.07| 0.3414285714285714|
|2013-08-16|2013|    8| 16|     11804.0|  null|   GGBRH16|    GGBR /EJ|      0.01|       0.25|      0.22|      0.32|  

### ETL World Bank

The `wbgapi` package is the official Python/R library to interact with the [World Bank open data](https://data.worldbank.org/),
it is useful to easily retrieve data, instead of download it ourselves.

The purpose is to fetch data from Brazil, then it will be possible to compare the stock market and the real economy.

In [31]:
# Metrics related to gdp
wb.series.info(q='gdp')

# Metrics related to debt, and so on
wb.series.info(q='debt')

id,value
DT.DOD.DECT.CD,"External debt stocks, total (DOD, current US$)"
DT.DOD.DECT.GN.ZS,External debt stocks (% of GNI)
DT.DOD.DLXF.CD,"External debt stocks, long-term (DOD, current US$)"
DT.DOD.DPNG.CD,"External debt stocks, private nonguaranteed (PNG) (DOD, current US$)"
DT.DOD.DPPG.CD,"External debt stocks, public and publicly guaranteed (PPG) (DOD, current US$)"
DT.DOD.DSTC.CD,"External debt stocks, short-term (DOD, current US$)"
DT.DOD.DSTC.IR.ZS,Short-term debt (% of total reserves)
DT.DOD.DSTC.XP.ZS,"Short-term debt (% of exports of goods, services and primary income)"
DT.DOD.DSTC.ZS,Short-term debt (% of total external debt)
DT.DOD.PVLX.CD,Present value of external debt (current US$)


In [32]:
metric_to_schema = {
    'GC.DOD.TOTL.GD.ZS': 'debt',
    'GC.XPN.TOTL.CN': 'total_expense',
    'NY.GDP.MKTP.KD.ZG': 'gdp_annual_growth',
    'NY.GDP.MKTP.CD': 'gdp',
    'NY.GDP.PCAP.CD': 'gdp_per_capita',
    'SP.POP.TOTL': 'population',
    'SP.DYN.LE00.IN': 'life_expectancy',
    'GC.XPN.TOTL.GD.ZS': 'expense_per_gdp',
    'FI.RES.TOTL.CD': 'total_reserves',
    'SE.ADT.LITR.ZS': 'pop_literacy_rate',
    'SE.XPD.TOTL.GD.ZS': 'expenditure_education_per_gdp',
    'BX.KLT.DINV.CD.WD': 'foreign_investment',
}

In [33]:
# Load data into pandas
country = 'BRA'
values = []
for key, value in metric_to_schema.items():
    df = wb.data.DataFrame(key, economy=country, columns='time', numericTimeKeys=True).dropna(axis=1, how='all')
    for data in df.items():
        values.append((data[0], value, data[1][country]))
economy_df = pd.DataFrame(data=values, columns=['year', 'table', 'value'])
economy_df = spark.createDataFrame(economy_df)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [34]:
# Performing cleaning tasks here




### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

### Trading Table

| Feature        | Description  |
| ------------- |:-------------:|
| date         | date that stock is being traded                      |
| year         | year that stock is being traded                      |
| month        | month that stock is being traded                     |
| day          | day that stock is being traded                       |
| money_volume | total amount of money in all transactions            |
| volume       | total number traded stocks                           |
| stock_code   | unique stock code                                    |
| company_name | name of the company that is represented by the stock |
| price_open   | price when trading opened                            |
| price_close  | price when trading closed                            |
| price_mean   | price mean of the stock                              |
| price_high   | price high of the stock                              |
| price_low    | price low of the stock                               |
| variation    | the variation between price close and price open     |

### Economy Table

| Feature        | Description  |
| ------------- |:-------------:|
| year  | the year the statics was collected           |
| table | the statics itself, like, GDP, debt over GDP |
| value | the value of the statics                     |

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [35]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [36]:
# Perform quality checks here
def quality_check(df_quality, name):
    """
    Count checks on table to ensure completeness of data.
    :param df_quality: spark dataframe to check counts on
    :param name: name of the dataframe
    """

    total_count = df_quality.count()

    if total_count == 0:
        print(f"Data quality check failed for {name} with zero records!")
    else:
        print(f"Data quality check passed for {name} with {total_count} records.")
    return 0


quality_check(economy_df, "economy")
quality_check(trading_df, "trading")

Data quality check passed for economy with 476 records.
Data quality check passed for trading with 378915 records.


0

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.